In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM,Dense, Dropout, BatchNormalization
from keras.preprocessing.sequence import TimeseriesGenerator

%matplotlib inline  




ModuleNotFoundError: No module named 'yfinance'

In [27]:
# df = yf.download('AMZN',start='2016-06-01', interval='1d',  end='2021-06-01',progress=False)

# df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
# df["Date"] = df.index
# df.head()
 
# #data.plot(figsize=(10,10))

In [3]:
# Load data to a df using pandas
import sqlite3
import pandas as pd
database_path = "./stocktrx.db"

# Create your connection.
cnx = sqlite3.connect(database_path)

df = pd.read_sql_query("SELECT * FROM stocks", cnx)
df.head()

,TrxDate,Ticker,Open,High,Low,Close,AdjClose,Volume
0,2017-07-19 00:00:00,AAPL,37.619999,37.855000,37.487499,37.755001,35.945583,83692000.0
1,2017-07-19 00:00:00,BA,209.429993,210.929993,208.990005,210.880005,198.707397,2258400.0
2,2017-07-19 00:00:00,DAL,53.770000,54.340000,53.099998,53.790001,50.100189,10733000.0
3,2017-07-19 00:00:00,GE,25.817308,25.990385,25.807692,25.903847,24.509535,25784096.0
4,2017-07-19 00:00:00,GOOG,967.840027,973.039978,964.030029,970.890015,970.890015,1224500.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TrxDate   100 non-null    object 
 1   Ticker    100 non-null    object 
 2   Open      100 non-null    float64
 3   High      100 non-null    float64
 4   Low       100 non-null    float64
 5   Close     100 non-null    float64
 6   AdjClose  100 non-null    float64
 7   Volume    100 non-null    float64
dtypes: float64(6), object(2)
memory usage: 6.4+ KB


In [5]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

# date_train = df['Date'][:split]
# date_test = df['Date'][split:]
date_train = df['TrxDate'][:split]
date_test = df['TrxDate'][split:]

print(len(close_train))
print(len(close_test))

80
20


In [6]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

NameError: name 'TimeseriesGenerator' is not defined

In [7]:
model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

NameError: name 'Sequential' is not defined

In [8]:
#Test
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Amazon Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

NameError: name 'model' is not defined

In [9]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)
print(forecast_dates)

NameError: name 'model' is not defined

In [10]:
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "Amazon Stock",
    # xaxis = {'title' : "Date"},
    xaxis = {'title' : "TrxDate"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

NameError: name 'go' is not defined